In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import wandb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import Dataset, DatasetDict, load_metric,load_dataset
import evaluate

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- We must tokenise the input before feeding to the model. We use the tokenizer from the pretrained model.
- The dataset must have the label column named as 'label', to be trained using Trainer API. It is preferable to keep the text column as 'text'. However since we are tokenising the text column, it can be named anything, as we just pass the tokenised columns input_ids and attention_mask to the model.

In [2]:
# for huggingface pipeline to give text labels instead of numbers
id2label = {0: "suicide", 1:"non-suicide"}
label2id = {"suicide":0, "non-suicide":1}

In [3]:
finetune_model_name = "PHR_Suicide_Prediction_Roberta_Cleaned_Light"

In [3]:
tokeniser = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2,label2id=label2id,id2label=id2label)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset = load_dataset("vibhorag101/roberta-base-suicide-prediction-phr")

In [ ]:
def tokeniseDataset(dataset):
    return(tokeniser(dataset["text"],padding="max_length",truncation=True))

def convertLabel2ID(dataset):
    dataset['label'] = label2id[dataset['label']]
    return dataset
    
dataset = dataset.map(convertLabel2ID) 
tokenisedDataset = dataset.map(tokeniseDataset,batched=True)

trainTokeniseDataset = tokenisedDataset["train"]
tempTokenisedDataset= tokenisedDataset["test"]
tempTokenisedDataset = tempTokenisedDataset.train_test_split(test_size=0.5)
testTokenisedDataset = tempTokenisedDataset["train"]
valTokenisedDataset = tempTokenisedDataset["test"]
# print(trainTokeniseDataset)
# print(valTokenisedDataset)
# print(testTokenisedDataset)

In [ ]:
def compute_metrics(eval_pred):
    metric_acc = evaluate.load("accuracy")
    metric_rec = evaluate.load("recall")
    metric_pre = evaluate.load("precision")
    metric_f1 = evaluate.load("f1")

    # here the model just give logits, labels are passed from test/val dataset
    logits, labels = eval_pred
    
    # for binary classification we can just take argmax of logits, but for multi-class classification we need to use softmax
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = metric_acc.compute(predictions=predictions, references=labels)
    recall = metric_rec.compute(predictions=predictions, references=labels)
    precision = metric_pre.compute(predictions=predictions, references=labels)
    f1 = metric_f1.compute(predictions=predictions, references=labels)

    return {"accuracy": accuracy, "recall": recall, "precision": precision, "f1": f1}


In [ ]:
wandb.login()
wandb.init(project="huggingface", entity="vibhor20349", name=finetune_model_name)

In [ ]:
training_args = TrainingArguments(
    output_dir=finetune_model_name,
    report_to = 'wandb',
    learning_rate=2e-5, # recommended in roberta paper
    num_train_epochs=3, #recommended in bert paper
    per_device_train_batch_size=16, # recommended in roberta paper
    per_device_eval_batch_size=16, # recommended in roberta paper
    evaluation_strategy="steps",
    eval_steps = 1000,
    save_strategy="steps",
    save_steps=1000,
    load_best_model_at_end=True,
    push_to_hub=True #to clone the training repo just before starting to train, so push to hub works
)

In [ ]:
## Here tokeniser is passed to the trainer, but it is for only pushing to hub and not for tokenising the dataset
trainer = Trainer(
    model= model,
    args=training_args,
    train_dataset=trainTokeniseDataset,
    eval_dataset=valTokenisedDataset,
    compute_metrics=compute_metrics,
    tokenizer=tokeniser,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3,early_stopping_threshold=0.001)]
)

In [ ]:
# print(trainer.predict(testTokenisedDataset).metrics)
trainer.evaluate(eval_dataset=testTokenisedDataset)

In [ ]:
trainer.train()
#trainer.train(resume_from_checkpoint=True)

In [ ]:
trainer.save_model(finetune_model_name)

In [ ]:
trainer.push_to_hub(finetune_model_name)

In [6]:
### IF we want to make predictions
input_text = "Your input text goes here."
inputs = tokeniser(input_text, return_tensors="pt")

# Make predictions
with torch.no_grad():
    ## BERT expects the keyword agruments "token_type_ids" and "attention_mask" in input.
    # so we convert inputs dictionary to keyword arguments using ** before passing to the model
    outputs = model(**inputs)
    ## output of Bert contains logits for each class, pooled output and hidden states and attentions

# Extract the predicted logits(raw values for each class)
logits = outputs.logits
print(logits)

## logits are just raw values for each class. To get probabilities we use softmax
probabilities = torch.nn.functional.softmax(logits, dim=1)
predicted_class = torch.argmax(probabilities, dim=1).item()

print(f"Predicted class: {predicted_class}")


tensor([[-0.1621, -0.1936]])
Predicted class: 0
